In [0]:
import pandas as pd
import numpy as np
import numpy as np
import sqlite3

In [0]:

file_path = "/Workspace/Users/joaovab@al.insper.edu.br/Hackathon_Terra_Signal_grupo_1/data/inference.csv"
df = pd.read_csv(file_path)
df.info()

In [0]:
filtro_vazios = df[df['TotalCharges'].isna()]
df['tenure'] = pd.to_numeric(df['tenure'], errors='coerce')
df['TotalCharges'] = df['TotalCharges'].fillna(df['tenure'] * df['MonthlyCharges'])
df.dropna(subset=['TotalCharges'], inplace=True)

In [0]:
colunas = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'MultipleLines']
for col in colunas:
    df[col] = df[col].replace({'No internet service': 'No', 'No phone service': 'No'})

df['PhoneService'] = df['PhoneService'].str.capitalize()
df.to_csv('../data/inference_clean.csv', index=False)

In [0]:
spark_df = spark.createDataFrame(df)
nome_tabela_uc = "workspace.churn_zero.inference_clean"
spark_df.write.format("delta").mode("overwrite").saveAsTable(nome_tabela_uc)

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.churn_zero.inference_silver AS
SELECT 
    *,
    
    -- 1. Sentimento
    ai_analyze_sentiment(CustomerFeedback) as sentiment_label,
    
    -- 2. Tópico (Labels ajustados para precisão)
   ai_gen(
            'Atue como um Analista de Retenção Sênior. Classifique o feedback em UMA categoria: 
        [sales_opportunity, lost_sales_opportunity, high_price, internet_connection, billing_issues, competitor_offer, contract_terms, personal_reasons, customer_service, sales_misunderstanding, general_satisfaction, service_quality, other].

        PERFIL:
        - Feedback: "' || CustomerFeedback || '"
        - Tem Internet: ' || InternetService || '
        
        REGRAS OBRIGATÓRIAS:
        1. OPORTUNIDADE (Priority Check): 
           - Se "Tem Internet" for "No" E o texto disser "wish that internet was included", "wish they offered internet", classifique como "sales_opportunity".
           - POREM, se o texto deixar claro que o cliente JÁ TOMOU A DECISÃO DE SAIR por causa disso ("ultimately decided to switch", "cancelled service due to lack of internet"), classifique como "lost_sales_opportunity".
        
        2. TÉCNICO: Se falar de "slow", "speed", "outage", "reliability", USE "internet_connection". NÃO use "service_quality" genericamente.
        3. VENDA ERRADA: Se "Tem Internet" for "No" mas ele reclamar que a conexão dele (da nossa empresa) está lenta, use "sales_misunderstanding".
        4. PREÇO: "Caro" = high_price. "Concorrente melhor" = competitor_offer.
        
        SAÍDA: Apenas o nome da categoria.'
        ) as feedback_topic,

    -- 3. Score Numérico
    CASE 
        WHEN ai_analyze_sentiment(CustomerFeedback) = 'positive' THEN 1 
        ELSE 0 
    END as sentiment_score

FROM workspace.churn_zero.inference_clean;

In [0]:
df = spark.sql("SELECT * FROM workspace.churn_zero.history_silver")
df.toPandas().to_csv("/Workspace/Users/joaovab@al.insper.edu.br/Hackathon_Terra_Signal_grupo_1/data/inference_silver.csv")